In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

tff.backends.reference.set_reference_context()

In [ ]:
import sys
sys.path.append("../")
from federated.data.data_preprocessing import load_data, create_unbalanced_data

In [ ]:
NUMBER_OF_CLIENTS=5
CLIENT_EPOCHS=10
BATCH_SIZE=32
NUM_EXAMPLES_PER_USER = 20_000

In [ ]:
train_client_data, test_client_data = load_data(normalized=True, data_selector=create_unbalanced_data)

In [ ]:
def get_data_for_ecg(source, client):
    output_sequence = []
    data = source[f"client_{client}"]
    all_samples = [i for i, y in enumerate(data[1])]
    for i in range(0, min(len(all_samples), NUM_EXAMPLES_PER_USER), BATCH_SIZE):
        batch_samples = all_samples[i:i + BATCH_SIZE]
        output_sequence.append({
            "x": np.array([data[0][i] for i in batch_samples], dtype=np.float32),
            "y": np.array([np.where(data[1][i]==1)[0][0] for i in batch_samples], dtype=np.int32)
        })
    return output_sequence

federated_train_data = [get_data_for_ecg(train_client_data, client) for client in range(1, 6)]

federated_test_data = [get_data_for_ecg(train_client_data, client) for client in range(1, 6)]

In [ ]:
BATCH_SPEC = collections.OrderedDict(
    x=tf.TensorSpec(shape=[None, 186], dtype=tf.float32),
    y=tf.TensorSpec(shape=[None], dtype=tf.int32))
BATCH_TYPE = tff.to_type(BATCH_SPEC)

str(BATCH_TYPE)

In [ ]:
MODEL_SPEC = collections.OrderedDict(
    weights=tf.TensorSpec(shape=[186, 5], dtype=tf.float32),
    bias=tf.TensorSpec(shape=[5], dtype=tf.float32))
MODEL_TYPE = tff.to_type(MODEL_SPEC)

print(MODEL_TYPE)

In [ ]:
@tf.function
def forward_pass(model, batch):
    predicted_y = tf.nn.softmax(
        tf.matmul(batch['x'], model['weights']) + model['bias'])
    return -tf.reduce_mean(
        tf.reduce_sum(
            tf.one_hot(batch['y'], 5) * tf.math.log(predicted_y), axis=[1]))

@tff.tf_computation(MODEL_TYPE, BATCH_TYPE)
def batch_loss(model, batch):
    return forward_pass(model, batch)

In [ ]:
initial_model = collections.OrderedDict(
    weights=np.zeros([186, 5], dtype=np.float32),
    bias=np.zeros([5], dtype=np.float32))

sample_batch = federated_train_data[4][-1]

batch_loss(initial_model, sample_batch)

In [ ]:
@tff.tf_computation(MODEL_TYPE, BATCH_TYPE, tf.float32)
def batch_train(initial_model, batch, learning_rate):
    # Define a group of model variables and set them to `initial_model`. Must
    # be defined outside the @tf.function
    model_vars = collections.OrderedDict([
        (name, tf.Variable(name=name, initial_value=value))
        for name, value in initial_model.items()
    ])
    optimizer = tf.keras.optimizers.SGD(learning_rate)

    @tf.function
    def _train_on_batch(model_vars, batch):
        # Perform one step gradient descent using loss from `batch_loss`.
        with tf.GradientTape() as tape:
            loss = forward_pass(model_vars, batch)
        grads = tape.gradient(loss, model_vars)
        optimizer.apply_gradients(
            zip(tf.nest.flatten(grads), tf.nest.flatten(model_vars)))
        return model_vars

    return _train_on_batch(model_vars, batch)

In [ ]:
LOCAL_DATA_TYPE = tff.SequenceType(BATCH_TYPE)

@tff.federated_computation(MODEL_TYPE, tf.float32, LOCAL_DATA_TYPE)
def local_train(initial_model, learning_rate, all_batches):

    # Mapping function to apply to each batch
    @tff.federated_computation(MODEL_TYPE, BATCH_TYPE)
    def batch_fn(model, batch):
        return batch_train(model, batch, learning_rate)
    
    return tff.sequence_reduce(all_batches, initial_model, batch_fn)

In [ ]:
locally_trained_model = local_train(initial_model, 0.1, federated_train_data[4])

In [ ]:
@tff.federated_computation(MODEL_TYPE, LOCAL_DATA_TYPE)
def local_eval(model, all_batches):
    return tff.sequence_sum(
        tff.sequence_map(
            tff.federated_computation(lambda b: batch_loss(model, b), BATCH_TYPE),
            all_batches))

In [ ]:
print('initial_model loss =', local_eval(initial_model,
                                          federated_train_data[4]))

print('locally_trained_model loss =',
      local_eval(locally_trained_model, federated_train_data[4]))

In [ ]:
print('initial_model loss =', local_eval(initial_model,
                                          federated_train_data[0]))

print('locally_trained_model loss =',
      local_eval(locally_trained_model, federated_train_data[0]))

In [ ]:
SERVER_MODEL_TYPE = tff.type_at_server(MODEL_TYPE)
CLIENT_DATA_TYPE = tff.type_at_clients(LOCAL_DATA_TYPE)

In [ ]:
@tff.federated_computation(SERVER_MODEL_TYPE, CLIENT_DATA_TYPE)
def federated_eval(model, data):
    return tff.federated_mean(
        tff.federated_map(local_eval, [tff.federated_broadcast(model), data]))

In [ ]:
print('initial_model loss =', federated_eval(initial_model,
                                          federated_train_data))

print('locally_trained_model loss =',
      federated_eval(locally_trained_model, federated_train_data))

In [ ]:
SERVER_FLOAT_TYPE = tff.type_at_server(tf.float32)

@tff.federated_computation(
    SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE, CLIENT_DATA_TYPE)
def grab_all_models(model, learning_rate, data):
  return tff.federated_collect(
      tff.federated_map(
          local_train,
          [tff.federated_broadcast(model),
           tff.federated_broadcast(learning_rate),
           data]))

@tff.federated_computation(SERVER_MODEL_TYPE, SERVER_FLOAT_TYPE,
                           CLIENT_DATA_TYPE)
def federated_train(model, learning_rate, data):
    return tff.federated_mean(
        tff.federated_map(local_train, [
            tff.federated_broadcast(model),
            tff.federated_broadcast(learning_rate), data
        ]))

In [ ]:
keras_model = tf.keras.models.load_model("../history/logdir/fedavg_linear_dp_v2")
weights = keras_model.layers[1].get_weights()[0]
biases = keras_model.layers[1].get_weights()[1]

print(weights.shape)
print(biases.shape)

In [ ]:
model = collections.OrderedDict(
    weights=weights,
    bias=biases)

In [ ]:
models = grab_all_models(model, 0.1, federated_train_data)

In [ ]:
import matplotlib.pyplot as plt

illness = ["Normal beats", "Supraven.", "Ventricular", "Fusion", "Unknown"]
#fig, axs = plt.subplots(5, 2, figsize=(20,20))

plt.figure(figsize=(20, 20))

for i in range(5):
    model = models[i]
    plt.subplot(3,3,i+1)
    plt.plot(model["weights"][:, 1])
    plt.title(f"Client {i+1}, Supraventricular Beats")

plt.savefig(f"images/weight_extraction_supra_dp.pdf", bbox_inches='tight')